In [5]:
!pip install -q faiss-gpu==1.7.2
!pip install pyserini==0.16
!pip install -q pytrec_eval


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      Dependence list: ['pybind11<2.6.2', 'psutil', "numpy>=1.10.0,<1.17 ; python_version=='2.7'", "numpy>=1.10.0 ; python_version>='3.5'"]
      /sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/setuptools/dist.py:757: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      /sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/site-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requireme

In [ ]:
# = Step 1) download corpus
!wget https://zenodo.org/records/6149599/files/data/wikipedia_split/full_wiki_segments.tsv -O corpus/TopiOCQA/full_wiki_segments.tsv

In [4]:
# = Step 2) Convert corpus to pyserini file
!python component0_preprocessing/bm25_corpus_indexing.py \
    --wiki_file "corpus/TopiOCQA/full_wiki_segments.tsv" \
    --output_file "corpus/TopiOCQA/full_wiki_segments_pyserini_format.jsonl"

/bin/bash: python: command not found


In [ ]:
# = Step 3) Index corpus using pyserini
!python -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 20 -input "corpus/TopiOCQA" -index "corpus/indexes" -storePositions -storeDocvectors -storeRaw

In [ ]:
!python component3_retriever/bm25/evaluation.py \
    --index_dir_path "corpus/indexes" \
    --result_qrel_path "component3_retriever/results" \
    --gold_qrel_path "component3_retriever/data/topiocqa/dev/qrel_gold.trec" \
    --dataset_name "topiocqa" \
    --query_format "original" \
    --seed 42